# Neighbourhood indicators

### Prepare environment and connections

In [1]:
library('RPostgreSQL')
library('magrittr')

Loading required package: DBI


In [2]:
conn <- dbConnect("PostgreSQL", dbname = 'base_data', host = 'localhost', user = 'postgres', password = 'asd')

#### Prepare the queries to indicators with radius

In [3]:
areas_query <- "WITH barea AS (
  SELECT a.id, SUM(b_area) AS areatot, SUM(b_floorsqm) AS floortot
  FROM cRADIUSm a, bd b
  WHERE ST_Intersects(a.geom, b.geom)
  AND a.id <> b.id
  GROUP BY a.id
)
UPDATE bd SET de_ar_RADIUSr = areatot,
              de_sq_RADIUSr = floortot
          FROM barea
          WHERE bd.id = barea.id;"

nb_build_query <- "WITH nb_bd AS (
  SELECT a.id, COUNT(b.geom) AS btot
  FROM cRADIUSm a, bd b
  WHERE ST_Intersects(a.geom, b.geom)
  AND a.id <> b.id
  GROUP BY a.id
)
UPDATE bd SET de_bd_RADIUSr = btot
          FROM nb_bd
          WHERE bd.id = nb_bd.id;"

deadend_query <- "WITH sel AS (
  SELECT id, edge_id, start_node AS st_n, end_node AS end_n
  FROM rd_topo.edge a, (SELECT * FROM cRADIUSm WHERE id = GID) b
 WHERE ST_Intersects (a.geom, b.geom)
), eid AS (
  SELECT DISTINCT node_id, geom
  FROM rd_topo.node a, sel
  WHERE node_id IN (SELECT st_n FROM sel)
   OR node_id IN (SELECT end_n FROM sel)
), sel_nodes AS (
  SELECT COUNT(*) AS all_n
  FROM eid
), dist_sn AS (
  SELECT COUNT(DISTINCT st_n) AS dst_n
  FROM sel
), dist_en AS (
  SELECT COUNT(DISTINCT end_n) AS dend_n
  FROM sel
), out_nodes AS (
  SELECT COUNT(*) AS out_n
  FROM (SELECT geom FROM cRADIUSm WHERE id = GID) a, eid b
  WHERE NOT ST_Intersects(a.geom, b.geom)
), dnd AS (
  SELECT DISTINCT id, (all_n - dst_n) + (all_n - dend_n) - out_n AS deadend
  FROM sel, sel_nodes, dist_sn, dist_en, out_nodes
)
UPDATE bd SET net_de_RADIUSr = CASE WHEN deadend < 0 THEN 0
                                 ELSE deadend
                            END
          FROM dnd
          WHERE bd.id = dnd.id;"

int_query <- "WITH in_nodes AS (
  SELECT id, COUNT(a.*) AS tot
  FROM rd_topo.node a, cRADIUSm b
  WHERE ST_Intersects(a.geom, b.geom)
  GROUP BY id
)
UPDATE bd SET net_int_RADIUSr = tot
          FROM in_nodes
          WHERE bd.id = in_nodes.id;"

length_query <- "WITH road_net AS (
  SELECT b.id, ST_Intersection(a.geom, b.geom) AS geom
  FROM rd a, cRADIUSm b 
  WHERE ST_Intersects(a.geom, b.geom)
), leng_rn AS (
  SELECT id, SUM(ST_Length(geom)) AS ltot
  FROM road_net
  GROUP BY id
)
UPDATE bd SET net_lgt_RADIUSr = ltot
          FROM leng_rn
          WHERE bd.id = leng_rn.id;"

This function is to change the query string according to the parameters

In [4]:
sub_in_query <- function(gid, radius, query) {
    q <- gsub('GID', gid, query) %>%
         gsub('RADIUS', radius, .)
    return(q)
}

## Densities and network indicators in x meters radius

Retrieve the IDs of bd

In [5]:
ids <- t(dbGetQuery(conn, "SELECT id FROM bd ORDER BY id;"))

Area and footprint

In [ ]:
for(rad in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, rad, areas_query)
    dbSendQuery(conn, q)
}

Number of buildings

In [ ]:
for(rad in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, rad, nb_build_query)
    dbSendQuery(conn, q)
}

Deadends

In [6]:
for(rad in 100){ #seq(100, 500, by = 200)) {
    for(id in ids) {
        q <- sub_in_query(id, rad, deadend_query)
        dbSendQuery(conn, q)
    }
}

Intersections

In [ ]:
for(rad in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, rad, int_query)
    dbSendQuery(conn, q)
}

Length of network

In [44]:
for(rad in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, rad, length_query)
    dbSendQuery(conn, q)
}

## Nearest neighbour (nn) and routing

### Nearest neighbour preparation

Nn queries

In [8]:
driving_vertices <- "WITH node_road AS (
  SELECT source, target
  FROM ways
  WHERE class_id NOT IN (114, 116, 117, 118, 119, 120, 122)
), road_nodes AS (
  SELECT id
  FROM ways_vertices_pgr
  WHERE id IN (SELECT DISTINCT source FROM node_road)
  OR id IN (SELECT DISTINCT target FROM node_road)
)
SELECT id, swigeom
INTO ways_vertices_drive
FROM ways_vertices_pgr
WHERE id IN (SELECT DISTINCT source FROM node_road)
OR id IN (SELECT DISTINCT target FROM node_road);"

nn_foot_query <- "WITH nearest AS (
  SELECT a.id AS batid, b.id AS node_id
  FROM bd a, ways_vertices_pgr b
  WHERE a.id = GID
  ORDER BY a.geom <-> b.swigeom LIMIT 1
)
UPDATE bd SET nn_foot = node_id
          FROM nearest
          WHERE bd.id = nearest.batid;"

nn_drive_query <- "WITH nearest AS (
  SELECT a.id AS batid, b.id AS node_id
  FROM bd a, ways_vertices_drive b
  WHERE a.id = GID
  ORDER BY a.geom <-> b.swigeom LIMIT 1
)
UPDATE bd SET nn_drive = node_id
          FROM nearest
          WHERE bd.id = nearest.batid;"

First, the network vertices are subsetted into a second table with only driveable roads taken into account

In [ ]:
dbSendQuery(conn, driving_vertices)

Then, for every building, the nearest node for both walking and driving routing is added as an attribute

In [ ]:
for(id in ids) {
    q <- sub_in_query(id, 0, nn_foot_query)
    dbSendQuery(conn, q)
}

for(id in ids) {
    q <- sub_in_query(id, 0, nn_drive_query)
    dbSendQuery(conn, q)
}

### Routing preparation

In [12]:
node_count_drive <- "SELECT COUNT(*) FROM pgr_drivingDistance('
    SELECT gid AS id,
        source,
        target,
        cost_s AS cost,
        reverse_cost_s AS reverse_cost
    FROM ways
    WHERE class_id NOT IN (114, 116, 117, 118, 119, 120, 122)',
    GID,
    120,
    true)"

node_count_walk <- "SELECT COUNT(*) FROM pgr_drivingDistance('SELECT gid As id, source, target,
      length_m AS cost
      FROM ways',
      GID,
      RADIUS,
      false)"

Then, for each routing elements and distance, a subset of all nodes from which it is possible to reach at least 3 nodes is created. This allows to create a polygon for the reachable area.

In [14]:
drive_nodes <- t(dbGetQuery(conn, 'SELECT DISTINCT nn_drive FROM bd ORDER BY nn_drive;'))
walk_nodes <- t(dbGetQuery(conn, 'SELECT DISTINCT nn_foot FROM bd ORDER BY nn_foot;'))

In [13]:
driveable_nodes <- numeric(0)
walkable_n_100m <- numeric(0)
walkable_n_300m <- numeric(0)
walkable_n_500m <- numeric(0)

In [ ]:
for(node in drive_nodes){
    q <- sub_in_query(node, 0, node_count_drive)
    nb_nodes <- dbGetQuery(conn, q)[[1]]
    if(nb_nodes > 2) {
        driveable_nodes <- c(driveable_nodes, id)
    }
}

for(node in walk_nodes){
    q <- sub_in_query(node, 100, node_count_walk)
    nb_nodes <- dbGetQuery(conn, q)[[1]]
    if(nb_nodes > 2) {
        walkable_n_100m <- c(walkable_n_100m, id)
    }
}

for(node in walk_nodes){
    q <- sub_in_query(node, 300, node_count_walk)
    nb_nodes <- dbGetQuery(conn, q)[[1]]
    if(nb_nodes > 2) {
        walkable_n_300m <- c(walkable_n_300m, id)
    }
}

for(node in walk_nodes){
    q <- sub_in_query(node, 500, node_count_walk)
    nb_nodes <- dbGetQuery(conn, q)[[1]]
    if(nb_nodes > 2) {
        walkable_n_500m <- c(walkable_n_500m, id)
    }
}

### Routing areas for various distance and options

Queries

In [7]:
first_drive_query <- "WITH area AS (
  SELECT GID As node_id, ST_Transform(ST_SetSRID(pgr_pointsAsPolygon(
    $$SELECT di.seq AS id, ST_X(v.the_geom) AS x, ST_Y(v.the_geom) As y
    FROM pgr_drivingDistance(''SELECT gid As id, source, target,
      cost_s AS cost, reverse_cost_s AS reverse_cost
      FROM ways
      WHERE class_id NOT IN (114, 116, 117, 118, 119, 120, 122)'',
      GID,
      120,
      true
  ) AS di
  INNER JOIN ways_vertices_pgr AS v ON di.node = v.id$$
  ), 4326), 21781) AS geom
)
SELECT * INTO r_drive_area
FROM area;"

r_drive_query <- "WITH area AS (
  SELECT GID As node_id, ST_Transform(ST_SetSRID(pgr_pointsAsPolygon(
    $$SELECT di.seq AS id, ST_X(v.the_geom) AS x, ST_Y(v.the_geom) As y
    FROM pgr_drivingDistance(''SELECT gid As id, source, target,
      cost_s AS cost, reverse_cost_s AS reverse_cost
      FROM ways
      WHERE class_id NOT IN (114, 116, 117, 118, 119, 120, 122)'',
      GID,
      120,
      true
  ) AS di
  INNER JOIN ways_vertices_pgr AS v ON di.node = v.id$$
  ), 4326), 21781) AS geom
)
INSERT INTO r_drive_area(node_id, geom)
SELECT node_id, geom
FROM area;"

foot_dist_f_query <- "WITH area AS (
  SELECT GID As node_id, ST_Transform(ST_SetSRID(pgr_pointsAsPolygon(
    $$SELECT di.seq AS id, ST_X(v.the_geom) AS x, ST_Y(v.the_geom) As y
    FROM pgr_drivingDistance(''SELECT gid As id, source, target,
      length_m AS cost
      FROM ways'',
      GID,
      RADIUS,
      false
  ) AS di
  INNER JOIN ways_vertices_pgr AS v ON di.node = v.id$$
  ), 4326), 21781) AS geom
)
SELECT * INTO r_walking_RADIUSm
FROM area;"

foot_dist_query <- "WITH area AS (
  SELECT GID As node_id, ST_Transform(ST_SetSRID(pgr_pointsAsPolygon(
    $$SELECT di.seq AS id, ST_X(v.the_geom) AS x, ST_Y(v.the_geom) As y
    FROM pgr_drivingDistance(''SELECT gid As id, source, target,
      length_m AS cost
      FROM ways'',
      GID,
      RADIUS,
      false
  ) AS di
  INNER JOIN ways_vertices_pgr AS v ON di.node = v.id$$
  ), 4326), 21781) AS geom
)
INSERT INTO r_walking_RADIUSm(node_id, geom)
SELECT node_id, geom
FROM area;"

In [ ]:
for(node in driveable_nodes) {
    if(node == driveable_nodes[1]) {
        q <- sub_in_query(node, 0, first_drive_query)
        dbSendQuery(conn, q)
    } else {
        q <- sub_in_query(node, 0, r_drive_query)
        dbSendQuery(conn, q)  
    }
}

for(node in walkable_n_100m) {
    if(node == walkable_n_100m[1]) {
        q <- sub_in_query(node, 100, foot_dist_f_query)
        dbSendQuery(conn, q)
    } else {
        q <- sub_in_query(node, 100, foot_dist_query)
        dbSendQuery(conn, q)  
    }
}

for(node in walkable_n_300m) {
    if(node == walkable_n_300m[1]) {
        q <- sub_in_query(node, 300, foot_dist_f_query)
        dbSendQuery(conn, q)
    } else {
        q <- sub_in_query(node, 300, foot_dist_query)
        dbSendQuery(conn, q)  
    }
}

for(node in walkable_n_500m) {
    if(node == walkable_n_500m[1]) {
        q <- sub_in_query(node, 500, foot_dist_f_query)
        dbSendQuery(conn, q)
    } else {
        q <- sub_in_query(node, 500, foot_dist_query)
        dbSendQuery(conn, q)  
    }
}

Create indices on the new tables

In [54]:
index_query <- "CREATE INDEX GID ON RADIUS USING GIST (geom);"
analyze_query <- "VACUUM ANALYZE RADIUS"
index_walk_query <- "CREATE INDEX r_walking_GID_gix ON r_walking_RADIUSm USING GIST (geom);"
analyze_walk_query <- "VACUUM ANALYZE r_walking_RADIUSm;"

In [ ]:
dbSendQuery(conn, sub_in_query("r_driving_gix", "r_drive_area", index_query))
dbSendQuery(conn, sub_in_query(0, "r_drive_area", analyze_query))

In [58]:
for(dist in seq(100, 500, by = 200)) {
    dbSendQuery(conn, sub_in_query(dist, dist, index_walk_query))
    dbSendQuery(conn, sub_in_query(0, dist, analyze_walk_query))
}

### Routing indicators

Queries

In [21]:
areas_r_query <- "WITH barea AS (
  SELECT a.node_id, SUM(b_area) AS areatot, SUM(b_floorsqm) AS floortot
  FROM r_walking_RADIUSm a, bd b
  WHERE ST_Intersects(a.geom, b.geom)
  GROUP BY a.node_id
)
UPDATE bd SET de_ar_RADIUSw = areatot,
              de_sq_RADIUSw = floortot
          FROM barea
          WHERE bd.nn_foot = barea.node_id;"

nb_build_r_query <- "WITH nb_bd AS (
  SELECT a.node_id, COUNT(b.geom) AS btot
  FROM r_walking_RADIUSm a, bd b
  WHERE ST_Intersects(a.geom, b.geom)
  GROUP BY a.node_id
)
UPDATE bd SET de_bd_RADIUSw = btot
          FROM nb_bd
          WHERE bd.nn_foot = nb_bd.node_id;"

deadend_r_query <- "WITH sel AS (
  SELECT id, edge_id, start_node AS st_n, end_node AS end_n
  FROM rd_topo.edge a, (SELECT node_id AS id, geom FROM r_walking_RADIUSm WHERE node_id = GID) b
  WHERE ST_Intersects (a.geom, b.geom)
), eid AS (
  SELECT DISTINCT node_id, geom
  FROM rd_topo.node a, sel
  WHERE node_id IN (SELECT st_n FROM sel)
   OR node_id IN (SELECT end_n FROM sel)
), sel_nodes AS (
  SELECT COUNT(*) AS all_n
  FROM eid
), dist_sn AS (
  SELECT COUNT(DISTINCT st_n) AS dst_n
  FROM sel
), dist_en AS (
  SELECT COUNT(DISTINCT end_n) AS dend_n
  FROM sel
), out_nodes AS (
  SELECT COUNT(*) AS out_n
  FROM (SELECT geom FROM r_walking_RADIUSm WHERE node_id = GID) a, eid b
  WHERE NOT ST_Intersects(a.geom, b.geom)
), dnd AS (
  SELECT DISTINCT id, (all_n - dst_n) + (all_n - dend_n) - out_n AS deadend
  FROM sel, sel_nodes, dist_sn, dist_en, out_nodes
)
UPDATE bd SET net_de_RADIUSw = CASE WHEN deadend < 0 THEN 0
                                    ELSE deadend
                               END
          FROM dnd
          WHERE bd.nn_foot = dnd.id;"

int_r_query <- "WITH in_nodes AS (
  SELECT b.node_id AS id, COUNT(a.*) AS tot
  FROM rd_topo.node a, r_walking_RADIUSm b
  WHERE ST_Intersects(a.geom, b.geom)
  GROUP BY b.node_id
)
UPDATE bd SET net_int_RADIUSw = tot
          FROM in_nodes
          WHERE bd.nn_foot = in_nodes.id;"

length_r_query <- "WITH road_net AS (
  SELECT b.node_id AS id, ST_Intersection(a.geom, b.geom) AS geom
  FROM rd a, r_walking_RADIUSm b 
  WHERE ST_Intersects(a.geom, b.geom)
), leng_rn AS (
  SELECT id, SUM(ST_Length(geom)) AS ltot
  FROM road_net
  GROUP BY id
)
UPDATE bd SET net_lgt_RADIUSw = ltot
          FROM leng_rn
          WHERE bd.nn_foot = leng_rn.id;"

id_r_query <- "SELECT node_id FROM r_walking_RADIUSm ORDER BY node_id"


id_dd_query <- "SELECT DISTINCT(node_id) FROM r_drive_area ORDER BY node_id"

deadend_dd_query <- "WITH sel AS (
  SELECT id, edge_id, start_node AS st_n, end_node AS end_n
  FROM rd_topo.edge a, (SELECT node_id AS id, geom FROM r_drive_area WHERE node_id = GID) b
  WHERE ST_Intersects (a.geom, b.geom)
), eid AS (
  SELECT DISTINCT node_id, geom
  FROM rd_topo.node a, sel
  WHERE node_id IN (SELECT st_n FROM sel)
   OR node_id IN (SELECT end_n FROM sel)
), sel_nodes AS (
  SELECT COUNT(*) AS all_n
  FROM eid
), dist_sn AS (
  SELECT COUNT(DISTINCT st_n) AS dst_n
  FROM sel
), dist_en AS (
  SELECT COUNT(DISTINCT end_n) AS dend_n
  FROM sel
), out_nodes AS (
  SELECT COUNT(*) AS out_n
  FROM (SELECT geom FROM r_drive_area WHERE node_id = GID) a, eid b
  WHERE NOT ST_Intersects(a.geom, b.geom)
), dnd AS (
  SELECT DISTINCT id, (all_n - dst_n) + (all_n - dend_n) - out_n AS deadend
  FROM sel, sel_nodes, dist_sn, dist_en, out_nodes
)
UPDATE bd SET net_dd_de = CASE WHEN deadend < 0 THEN 0
                                    ELSE deadend
                               END
          FROM dnd
          WHERE bd.nn_drive = dnd.id;"

int_dd_query <- "WITH in_nodes AS (
  SELECT b.node_id AS id, COUNT(a.*) AS tot
  FROM rd_topo.node a, r_drive_area b
  WHERE ST_Intersects(a.geom, b.geom)
  GROUP BY b.node_id
)
UPDATE bd SET net_dd_int = tot
          FROM in_nodes
          WHERE bd.nn_drive = in_nodes.id;"

length_dd_query <- "WITH road_net AS (
  SELECT b.node_id AS id, ST_Intersection(a.geom, b.geom) AS geom
  FROM rd a, r_drive_area b 
  WHERE ST_Intersects(a.geom, b.geom)
  AND ST_IsValid(b.geom) = true
), leng_rn AS (
  SELECT id, SUM(ST_Length(geom)) AS ltot
  FROM road_net
  GROUP BY id
)
UPDATE bd SET net_dd_length = ltot
          FROM leng_rn
          WHERE bd.nn_drive = leng_rn.id;"

Areas

In [67]:
for(dist in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, dist, areas_r_query)
    dbSendQuery(conn, q)
}

Number of buildings

In [72]:
for(dist in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, dist, nb_build_r_query)
    dbSendQuery(conn, q)
}

Deadends

In [ ]:
for(dist in seq(100, 500, by = 200)) {
    ids_nodes <- t(dbGetQuery(conn, sub_in_query(0, dist, id_r_query)))
    for(id in ids_nodes) {
        q <- sub_in_query(id, dist, deadend_r_query)
        dbSendQuery(conn, q)
    }
}

Intersections

In [79]:
for(rad in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, rad, int_r_query)
    dbSendQuery(conn, q)
}

Length of network

In [83]:
for(rad in seq(100, 500, by = 200)) {
    q <- sub_in_query(0, rad, length_r_query)
    dbSendQuery(conn, q)
}

### Driving distance

In [12]:
dd_ids <- t(dbGetQuery(conn, id_dd_query))

In [14]:
for(id in dd_ids) {
    q <- sub_in_query(id, 0, deadend_dd_query)
    dbSendQuery(conn, q)
}

In [19]:
dbSendQuery(conn, int_dd_query)

<PostgreSQLResult>

In [22]:
dbSendQuery(conn, length_dd_query)

<PostgreSQLResult>